THIS ONE WORKS, by preprocessing each column independently into a tensor we can challenge it. The problem is that there is no computing power (both in local and google colab)

In [2]:
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import tqdm as notebook_tqdm
 
import numpy as np
import pandas as pd
import tensorflow as tf
 
from typing import Dict, Text
from tensorflow.keras import layers
import pprint

/home/pablo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-04 19:24:02.643985: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-04 19:24:02.644042: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-04 19:24:02.651503: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-04 19:24:04.079139: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is op

In [4]:
#we get the customers and article data
# transactions = pd.read_csv('../00 - Data/transactions_train/transactions_train.csv')
transactions_short = pd.read_csv('../00 - Data/transactions_train/short_transactions.csv')
# transactions.head()

To build the preprocessing model, start by building a set of symbolic tf.keras.Input objects, matching the names and data-types of the CSV columns.

In [5]:
len(transactions_short)

1179208

In [9]:
transactions_features = transactions_short.head(10000).copy()
transactions_labels = transactions_features

10000


In [10]:
inputs = {}

for name, column in transactions_features.items():
  dtype = column.dtype
  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

inputs

{'t_dat': <KerasTensor: shape=(None, 1) dtype=string (created by layer 't_dat')>,
 'customer_id': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'customer_id')>,
 'article_id': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'article_id')>,
 'price': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'price')>,
 'sales_channel_id': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'sales_channel_id')>}

The first step in your preprocessing logic is to concatenate the numeric inputs together, and run them through a normalization layer:

In [12]:
numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = layers.Normalization()
norm.adapt(np.array(transactions_features[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)

all_numeric_inputs

<KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'normalization_1')>

In [14]:
preprocessed_inputs = [all_numeric_inputs]

In [15]:
for name, input in inputs.items():
  if input.dtype == tf.float32:
    continue

  lookup = layers.StringLookup(vocabulary=np.unique(transactions_features[name]))
  one_hot = layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())

  x = lookup(input)
  x = one_hot(x)
  preprocessed_inputs.append(x)

In [16]:
preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)

transactions_preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)

tf.keras.utils.plot_model(model = transactions_preprocessing , rankdir="LR", dpi=72, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [17]:
transactions_features_dict = {name: np.array(value) 
                         for name, value in transactions_features.items()}

In [18]:
features_dict = {name:values[:1] for name, values in transactions_features_dict.items()}
transactions_preprocessing(features_dict)

<tf.Tensor: shape=(1, 3397), dtype=float32, numpy=
array([[ 0.66861194, -0.60040575,  1.0149133 , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)>

In [19]:
def transactions_model(preprocessing_head, inputs):
  body = tf.keras.Sequential([
    layers.Dense(64),
    layers.Dense(1)
  ])

  preprocessed_inputs = preprocessing_head(inputs)
  result = body(preprocessed_inputs)
  model = tf.keras.Model(inputs, result)

  model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam())
  return model

transactions_model2 = transactions_model(transactions_preprocessing, inputs)

In [21]:
transactions_model2.fit(x=transactions_features_dict, y=transactions_features, epochs=10)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).